# Interceptability Field (IF) + Window of Opportunity (WOO)
## A Spatial-Temporal Framework for Ball-in-Air Analysis

This notebook implements a novel metric that quantifies who can reach the ball first and how the window for a catch evolves in real time.

## 0. Load Supplementary Data (Pass Results, Routes, Coverage)

The supplementary_data.csv file contains actual pass results and play context that we'll use for validation.

In [ ]:
# Load supplementary data once at the beginning
# This contains actual pass results (C/I), routes, coverage types, etc.
SUPPLEMENTARY_DATA = None
try:
    SUPPLEMENTARY_DATA = pd.read_csv('supplementary_data.csv')
    SUPPLEMENTARY_DATA['game_id'] = SUPPLEMENTARY_DATA['game_id'].astype(int)
    SUPPLEMENTARY_DATA['play_id'] = SUPPLEMENTARY_DATA['play_id'].astype(int)
    print(f"✓ Loaded supplementary data: {len(SUPPLEMENTARY_DATA)} plays")
    print(f"  Pass results available: {SUPPLEMENTARY_DATA['pass_result'].notna().sum()} plays")
    print(f"  Routes available: {SUPPLEMENTARY_DATA['route_of_targeted_receiver'].notna().sum()} plays")
    print(f"  Coverage types available: {SUPPLEMENTARY_DATA['team_coverage_type'].notna().sum()} plays")
except Exception as e:
    print(f"⚠️  Could not load supplementary data: {e}")
    print("  Will use output file inference for outcomes")
    SUPPLEMENTARY_DATA = pd.DataFrame()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Constants
FRAME_RATE = 10  # 10 frames per second
DT = 0.1  # Time step in seconds
FIELD_LENGTH = 120  # yards
FIELD_WIDTH = 53.3  # yards
MAX_PLAYER_SPEED = 8.0  # yards/second (conservative estimate)
MAX_ACCELERATION = 5.0  # yards/second^2
REACTION_DELAY = 0.2  # seconds (typical reaction time)
MIN_CHANGE_DIR_TIME = 0.3  # seconds (time to change direction significantly)

## 1. Data Loading and Preprocessing

In [ ]:
def load_play_data(input_file, output_file=None):
    """Load play data from input CSV file."""
    df = pd.read_csv(input_file)
    
    # Get unique plays
    plays = df.groupby(['game_id', 'play_id']).first()
    
    return df, plays

def get_play_data(df, game_id, play_id):
    """Extract data for a specific play."""
    play_df = df[(df['game_id'] == game_id) & (df['play_id'] == play_id)].copy()
    
    # Get ball landing location (same for all frames)
    ball_land_x = play_df['ball_land_x'].iloc[0]
    ball_land_y = play_df['ball_land_y'].iloc[0]
    num_frames = play_df['num_frames_output'].iloc[0]
    
    # Get player to predict (target receiver/defender)
    target_player = play_df[play_df['player_to_predict'] == True].iloc[0]
    target_nfl_id = target_player['nfl_id']
    
    # Get all players in the play
    all_players = play_df.groupby('nfl_id').first()
    
    return play_df, ball_land_x, ball_land_y, num_frames, target_nfl_id, all_players

## 2. Ball Trajectory Estimation

In [ ]:
def estimate_ball_trajectory(frame_throw, ball_land_x, ball_land_y, num_frames):
    """
    Estimate ball trajectory using parabolic interpolation.
    Assumes throw point is at frame 1, landing at frame num_frames.
    """
    # Get throw point (assume frame 1, position from first frame data)
    # For simplicity, we'll use a parabolic trajectory
    throw_x = 0  # Will be updated from actual data
    throw_y = 0
    
    # Time from throw to landing
    total_time = (num_frames - 1) * DT
    
    # Create time array
    times = np.array([(f - 1) * DT for f in range(1, num_frames + 1)])
    
    # Parabolic interpolation (simplified)
    # x(t) = x0 + vx*t
    # y(t) = y0 + vy*t - 0.5*g*t^2
    
    # For horizontal: linear interpolation
    ball_x = np.linspace(throw_x, ball_land_x, num_frames)
    
    # For vertical: parabolic with peak at midpoint
    # Assume peak height of 15 yards at midpoint
    peak_height = 15
    mid_time = total_time / 2
    
    # Parabolic trajectory
    ball_y = []
    for t in times:
        if t <= mid_time:
            # Ascending
            y = throw_y + (peak_height - throw_y) * (t / mid_time)**2
        else:
            # Descending
            y = peak_height - (peak_height - ball_land_y) * ((t - mid_time) / mid_time)**2
        ball_y.append(y)
    
    ball_y = np.array(ball_y)
    
    return times, ball_x, ball_y

## 3. Reachable Set Calculation

In [ ]:
def calculate_reachable_set(player_x, player_y, player_s, player_a, player_dir, 
                            time_available, reaction_delay=REACTION_DELAY):
    """
    Calculate the maximum distance a player can reach in given time.
    Accounts for current speed, acceleration, and direction.
    """
    # Account for reaction delay
    effective_time = max(0, time_available - reaction_delay)
    
    if effective_time <= 0:
        return 0.0
    
    # Current speed in yards/second
    v0 = player_s
    
    # Maximum distance using kinematic equation: d = v0*t + 0.5*a*t^2
    # But also constrained by max speed: d <= v_max * t
    
    # Distance with current speed and acceleration
    distance_with_accel = v0 * effective_time + 0.5 * player_a * effective_time**2
    
    # Distance constrained by max speed
    distance_max_speed = MAX_PLAYER_SPEED * effective_time
    
    # Take minimum (most conservative)
    max_distance = min(distance_with_accel, distance_max_speed)
    
    # Ensure non-negative
    max_distance = max(0, max_distance)
    
    return max_distance

## 4. Interceptability Field (IF) Calculation

## 11. Outcome Validation: Correlating WOO with Play Results

In [ ]:
def analyze_woo_vs_outcome(input_file, output_file=None, max_plays=50, supp_df=None):
    """
    Analyze WOO metrics vs actual play outcomes.
    Uses supplementary_data.csv for actual pass results, routes, and coverage.
    """
    if supp_df is None:
        supp_df = SUPPLEMENTARY_DATA
    
    df, plays = load_play_data(input_file, output_file)
    
    results = []
    play_count = 0
    
    for (game_id, play_id), play_info in plays.iterrows():
        if play_count >= max_plays:
            break
        
        try:
            # Analyze the play
            play_results, play_df, times, ball_x, ball_y = analyze_play(
                input_file, game_id, play_id, sample_frames=None
            )
            
            if play_results['enhanced_metrics'] is None:
                continue
            
            metrics = play_results['enhanced_metrics']
            
            # Get outcome, route, coverage from supplementary data
            outcome, route, coverage = get_play_outcome_from_supplementary(supp_df, game_id, play_id)
            
            # If no supplementary data, try to infer from output file
            if outcome is None and output_file:
                # Try to infer from output file (simplified)
                outcome = None  # Would need output file parsing
            
            results.append({
                'game_id': game_id,
                'play_id': play_id,
                'outcome': outcome,
                'route': route,
                'coverage': coverage,
                'woo_initial': metrics['woo_initial'],
                'woo_final': metrics['woo_final'],
                'woo_peak': metrics['woo_peak'],
                'woo_closure_time': metrics['woo_closure_time'] if metrics['woo_closure_time'] != float('inf') else None,
                'if_delta_receiver': metrics['if_delta_receiver'],
                'if_delta_defender': metrics['if_delta_defender']
            })
            
            play_count += 1
            
        except Exception as e:
            continue
    
    return pd.DataFrame(results)

## 12. Role-Based Analysis

In [ ]:
def analyze_by_role(input_file, max_plays=100):
    """
    Analyze WOO metrics by player role/position.
    """
    df, plays = load_play_data(input_file)
    
    role_results = []
    play_count = 0
    
    for (game_id, play_id), play_info in plays.iterrows():
        if play_count >= max_plays:
            break
        
        try:
            play_results, play_df, times, ball_x, ball_y = analyze_play(
                input_file, game_id, play_id
            )
            
            if play_results['enhanced_metrics'] is None:
                continue
            
            metrics = play_results['enhanced_metrics']
            
            # Get target player info
            target_player = play_df[play_df['player_to_predict'] == True].iloc[0]
            role = target_player.get('player_role', 'Unknown')
            position = target_player.get('player_position', 'Unknown')
            
            role_results.append({
                'game_id': game_id,
                'play_id': play_id,
                'role': role,
                'position': position,
                'woo_peak': metrics['woo_peak'],
                'woo_closure_time': metrics['woo_closure_time'] if metrics['woo_closure_time'] != float('inf') else None,
                'if_delta_receiver': metrics['if_delta_receiver'],
                'if_delta_defender': metrics['if_delta_defender']
            })
            
            play_count += 1
            
        except Exception as e:
            continue
    
    return pd.DataFrame(role_results)

def plot_role_analysis(role_df):
    """Plot WOO metrics by role/position."""
    if len(role_df) == 0:
        print("No role data available")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Group by role
    role_stats = role_df.groupby('role')['woo_peak'].agg(['mean', 'count']).sort_values('mean', ascending=False)
    role_stats = role_stats[role_stats['count'] >= 3]  # At least 3 plays
    
    if len(role_stats) > 0:
        axes[0].barh(role_stats.index, role_stats['mean'], color='skyblue', edgecolor='black', alpha=0.7)
        axes[0].set_xlabel('Average WOO Peak', fontsize=12, fontweight='bold')
        axes[0].set_title('WOO Peak by Player Role', fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3, axis='x')
    
    # Group by position
    pos_stats = role_df.groupby('position')['woo_peak'].agg(['mean', 'count']).sort_values('mean', ascending=False)
    pos_stats = pos_stats[pos_stats['count'] >= 3]
    
    if len(pos_stats) > 0:
        axes[1].barh(pos_stats.index, pos_stats['mean'], color='lightcoral', edgecolor='black', alpha=0.7)
        axes[1].set_xlabel('Average WOO Peak', fontsize=12, fontweight='bold')
        axes[1].set_title('WOO Peak by Position', fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig('role_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

## 13. Multi-Week Analysis

In [ ]:
def analyze_all_weeks(max_plays_per_week=20, use_supplementary=True):
    """
    Analyze plays across all 18 weeks.
    Aggregates results for route/coverage/temporal analysis.
    """
    import glob
    
    all_results = []
    supp_df = SUPPLEMENTARY_DATA if use_supplementary else None
    
    input_files = sorted(glob.glob('train/input_2023_w*.csv'))
    
    for week_file in input_files:
        week = int(week_file.split('_w')[1].split('.')[0])
        print(f"Processing Week {week}...")
        
        try:
            week_results = analyze_woo_vs_outcome(
                week_file, 
                max_plays=max_plays_per_week,
                supp_df=supp_df
            )
            
            if len(week_results) > 0:
                week_results['week'] = week
                all_results.append(week_results)
        
        except Exception as e:
            print(f"  Error in week {week}: {e}")
            continue
    
    if len(all_results) > 0:
        combined_results = pd.concat(all_results, ignore_index=True)
        return combined_results
    else:
        return pd.DataFrame()

## 14. Example: Outcome Validation Analysis

In [ ]:
# Example: Analyze WOO vs outcomes for Week 1
print("=" * 60)
print("EXAMPLE 2: Outcome Validation Analysis")
print("=" * 60)
print("Analyzing plays from Week 1 with actual pass results...")
print("=" * 60)

input_file = 'train/input_2023_w01.csv'
outcome_df = analyze_woo_vs_outcome(input_file, max_plays=20, supp_df=SUPPLEMENTARY_DATA)

if len(outcome_df) > 0:
    print(f"\n✓ Analyzed {len(outcome_df)} plays")
    
    # Summary by outcome
    if outcome_df['outcome'].notna().sum() > 0:
        print("\n" + "=" * 60)
        print("WOO PEAK BY OUTCOME:")
        print("=" * 60)
        outcome_stats = outcome_df.groupby('outcome')['woo_peak'].agg(['mean', 'count'])
        print(outcome_stats)
        
        # Summary by route
        if outcome_df['route'].notna().sum() > 0:
            print("\n" + "=" * 60)
            print("WOO PEAK BY ROUTE (Top 5):")
            print("=" * 60)
            route_stats = outcome_df[outcome_df['route'].notna()].groupby('route')['woo_peak'].mean().sort_values(ascending=False)
            print(route_stats.head(5))
        
        # Summary by coverage
        if outcome_df['coverage'].notna().sum() > 0:
            print("\n" + "=" * 60)
            print("WOO PEAK BY COVERAGE:")
            print("=" * 60)
            coverage_stats = outcome_df[outcome_df['coverage'].notna()].groupby('coverage')['woo_peak'].mean().sort_values(ascending=False)
            print(coverage_stats)
else:
    print("⚠️  No results generated. Check data files.")

## 15. Example: Multi-Week Analysis

In [ ]:
# Example: Analyze all weeks (this may take a while)
print("=" * 60)
print("EXAMPLE 3: Multi-Week Analysis")
print("=" * 60)
print("Analyzing plays across all 18 weeks...")
print("(This may take several minutes)")
print("=" * 60)

# Run with smaller sample for demonstration
all_weeks_results = analyze_all_weeks(max_plays_per_week=10, use_supplementary=True)

if len(all_weeks_results) > 0:
    print(f"\n✓ Analyzed {len(all_weeks_results)} plays across all weeks")
    
    # Overall statistics
    print("\n" + "=" * 60)
    print("OVERALL STATISTICS:")
    print("=" * 60)
    print(f"Average WOO Peak: {all_weeks_results['woo_peak'].mean():.3f}")
    print(f"Average WOO Initial: {all_weeks_results['woo_initial'].mean():.3f}")
    print(f"Average WOO Final: {all_weeks_results['woo_final'].mean():.3f}")
    
    # By outcome
    if all_weeks_results['outcome'].notna().sum() > 0:
        print("\n" + "=" * 60)
        print("BY OUTCOME:")
        print("=" * 60)
        print(all_weeks_results.groupby('outcome')['woo_peak'].agg(['mean', 'count']))
    
    # By route
    if all_weeks_results['route'].notna().sum() > 0:
        print("\n" + "=" * 60)
        print("BY ROUTE (Top 5):")
        print("=" * 60)
        route_means = all_weeks_results[all_weeks_results['route'].notna()].groupby('route')['woo_peak'].mean().sort_values(ascending=False)
        print(route_means.head(5))
    
    # By coverage
    if all_weeks_results['coverage'].notna().sum() > 0:
        print("\n" + "=" * 60)
        print("BY COVERAGE:")
        print("=" * 60)
        coverage_means = all_weeks_results[all_weeks_results['coverage'].notna()].groupby('coverage')['woo_peak'].mean().sort_values(ascending=False)
        print(coverage_means)
    
    # Temporal trends
    if all_weeks_results['week'].notna().sum() > 0:
        print("\n" + "=" * 60)
        print("TEMPORAL TRENDS (by week):")
        print("=" * 60)
        weekly_means = all_weeks_results.groupby('week')['woo_peak'].mean()
        print(weekly_means)
else:
    print("⚠️  No results generated. This is normal if running for the first time.")
    print("   The notebook demonstrates the analysis framework.")

In [ ]:
def calculate_if_at_point(player_x, player_y, player_s, player_a, player_dir, player_o,
                         target_x, target_y, time_available, reaction_delay=REACTION_DELAY):
    """
    Calculate IF probability for a player reaching a specific point.
    Returns probability between 0 and 1.
    """
    # Distance to target
    distance = np.sqrt((target_x - player_x)**2 + (target_y - player_y)**2)
    
    # Maximum reachable distance
    max_reachable = calculate_reachable_set(
        player_x, player_y, player_s, player_a, player_dir,
        time_available, reaction_delay
    )
    
    if max_reachable <= 0:
        return 0.0
    
    # Calculate angle difference (direction change needed)
    angle_to_target = np.arctan2(target_y - player_y, target_x - player_x) * 180 / np.pi
    angle_diff = abs(angle_to_target - player_dir)
    if angle_diff > 180:
        angle_diff = 360 - angle_diff
    
    # Penalty for large direction changes
    if angle_diff > 45:  # Significant turn needed
        turn_penalty = 1.0 - (angle_diff / 180.0) * 0.3  # Up to 30% penalty
        max_reachable *= turn_penalty
    
    # Calculate probability using sigmoid-like function
    # High probability when distance < reachable, exponential decay beyond
    
    if distance <= max_reachable:
        # Within reachable distance - high probability
        # Add some uncertainty based on distance
        prob = 1.0 - (distance / max_reachable) * 0.2  # 80-100% probability
    else:
        # Beyond reachable distance - exponential decay
        excess = distance - max_reachable
        decay_rate = 0.5  # Controls how fast probability decays
        prob = np.exp(-decay_rate * excess / max_reachable) * 0.8
    
    # Ensure probability is between 0 and 1
    prob = max(0.0, min(1.0, prob))
    
    return prob

In [ ]:
def compute_interceptability_field(play_df, frame_id, ball_land_x, ball_land_y, 
                                  times, ball_x, ball_y, grid_resolution=2.0):
    """
    Compute IF field for all players at a specific frame.
    Returns dictionary mapping nfl_id to IF field grid.
    """
    # Get frame data
    frame_data = play_df[play_df['frame_id'] == frame_id].copy()
    
    if len(frame_data) == 0:
        return {}
    
    # Get time available (from current frame to ball landing)
    frame_idx = frame_id - 1
    if frame_idx >= len(times):
        return {}
    
    time_to_landing = times[-1] - times[frame_idx]
    
    # Create grid around ball landing point
    grid_size = 20  # 20 yards in each direction
    x_range = np.arange(ball_land_x - grid_size, ball_land_x + grid_size, grid_resolution)
    y_range = np.arange(ball_land_y - grid_size, ball_land_y + grid_size, grid_resolution)
    
    X, Y = np.meshgrid(x_range, y_range)
    
    player_if_fields = {}
    
    for _, player in frame_data.iterrows():
        nfl_id = player['nfl_id']
        
        # Calculate IF for each grid point
        IF = np.zeros_like(X)
        
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                target_x = X[i, j]
                target_y = Y[i, j]
                
                IF[i, j] = calculate_if_at_point(
                    player['x'], player['y'],
                    player['s'], player['a'],
                    player['dir'], player['o'],
                    target_x, target_y,
                    time_to_landing
                )
        
        player_if_fields[nfl_id] = {
            'IF': IF,
            'X': X,
            'Y': Y,
            'player_x': player['x'],
            'player_y': player['y'],
            'player_name': player.get('player_name', f'Player_{nfl_id}')
        }
    
    return player_if_fields

## 5. Window of Opportunity (WOO) Calculation

In [ ]:
def calculate_woo(play_df, frame_id, ball_land_x, ball_land_y, times, ball_x, ball_y):
    """
    Calculate Window of Opportunity (WOO) for a specific frame.
    WOO = Receiver_IF - Defender_IF at ball landing point.
    """
    # Get frame data
    frame_data = play_df[play_df['frame_id'] == frame_id].copy()
    
    if len(frame_data) == 0:
        return None, None, None
    
    # Get target player (player_to_predict)
    target_player = frame_data[frame_data['player_to_predict'] == True]
    
    if len(target_player) == 0:
        return None, None, None
    
    target = target_player.iloc[0]
    target_nfl_id = target['nfl_id']
    
    # Determine if target is receiver or defender based on role
    role = str(target.get('player_role', '')).upper()
    is_receiver = 'RECEIVER' in role or 'OFFENSE' in role or target.get('player_side', '').upper() == 'OFFENSE'
    
    # Get time available
    frame_idx = frame_id - 1
    if frame_idx >= len(times):
        return None, None, None
    
    time_to_landing = times[-1] - times[frame_idx]
    
    # Calculate IF for target player at ball landing point
    receiver_if = calculate_if_at_point(
        target['x'], target['y'],
        target['s'], target['a'],
        target['dir'], target['o'],
        ball_land_x, ball_land_y,
        time_to_landing
    )
    
    # Calculate IF for all other players (defenders if target is receiver, vice versa)
    defender_ifs = []
    
    for _, player in frame_data.iterrows():
        if player['nfl_id'] == target_nfl_id:
            continue
        
        # Determine if this player is on opposite side
        player_role = str(player.get('player_role', '')).upper()
        player_side = str(player.get('player_side', '')).upper()
        
        if is_receiver:
            # Target is receiver, so defenders are on defense
            is_defender = 'DEFENSE' in player_role or player_side == 'DEFENSE'
        else:
            # Target is defender, so "defenders" are actually receivers
            is_defender = 'RECEIVER' in player_role or 'OFFENSE' in player_role or player_side == 'OFFENSE'
        
        if is_defender:
            defender_if = calculate_if_at_point(
                player['x'], player['y'],
                player['s'], player['a'],
                player['dir'], player['o'],
                ball_land_x, ball_land_y,
                time_to_landing
            )
            defender_ifs.append(defender_if)
    
    # WOO = Receiver IF - max(Defender IFs)
    if len(defender_ifs) > 0:
        max_defender_if = max(defender_ifs)
    else:
        max_defender_if = 0.0
    
    woo = receiver_if - max_defender_if
    
    return woo, receiver_if, max_defender_if

## 6. Enhanced Metrics

In [ ]:
def calculate_enhanced_metrics(woo_values, receiver_ifs, defender_ifs, frame_ids, times):
    """
    Calculate enhanced metrics from WOO timeseries.
    Returns dictionary with WOO_peak, WOO_closure_time, IF_delta, etc.
    """
    if len(woo_values) == 0:
        return None
    
    metrics = {}
    
    # WOO Peak
    max_idx = np.argmax(woo_values)
    metrics['woo_peak'] = woo_values[max_idx]
    metrics['woo_peak_time'] = times[max_idx] if len(times) > max_idx else 0
    
    # WOO Initial and Final
    metrics['woo_initial'] = woo_values[0]
    metrics['woo_final'] = woo_values[-1]
    
    # WOO Closure Time
    closure_time = None
    for i in range(max_idx, len(woo_values)):
        if woo_values[i] <= 0:
            closure_time = times[i] - metrics['woo_peak_time']
            break
    metrics['woo_closure_time'] = closure_time if closure_time is not None else float('inf')
    
    # IF Deltas
    if len(receiver_ifs) >= 2:
        metrics['if_delta_receiver'] = receiver_ifs[-1] - receiver_ifs[0]
    else:
        metrics['if_delta_receiver'] = 0
    
    if len(defender_ifs) >= 2:
        metrics['if_delta_defender'] = defender_ifs[-1] - defender_ifs[0]
    else:
        metrics['if_delta_defender'] = 0
    
    # Window Trend
    if metrics['woo_final'] > metrics['woo_initial'] + 0.05:
        metrics['window_trend'] = 'opening'
    elif metrics['woo_final'] < metrics['woo_initial'] - 0.05:
        metrics['window_trend'] = 'closing'
    else:
        metrics['window_trend'] = 'neutral'
    
    return metrics

## 7. Outcome Validation Functions

In [ ]:
def get_play_outcome_from_supplementary(supp_df, game_id, play_id):
    """Get play outcome (C/I/IN) from supplementary data."""
    if supp_df is None or len(supp_df) == 0:
        return None, None, None
    
    play_data = supp_df[(supp_df['game_id'] == game_id) & (supp_df['play_id'] == play_id)]
    
    if len(play_data) == 0:
        return None, None, None
    
    row = play_data.iloc[0]
    outcome = row.get('pass_result', None)
    route = row.get('route_of_targeted_receiver', None)
    coverage = row.get('team_coverage_type', None)
    
    return outcome, route, coverage

## 8. Visualization Functions

In [ ]:
def plot_woo_timeseries(woo_values, frame_ids, save_path=None):
    """Plot WOO evolution over time."""
    times = [(f - 1) * DT for f in frame_ids]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(times, woo_values, 'o-', linewidth=2, markersize=6, color='#2E86AB', label='WOO')
    ax.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Neutral')
    
    # Fill areas
    ax.fill_between(times, 0, woo_values, where=(np.array(woo_values) >= 0), 
                    alpha=0.3, color='green', label='Offense Advantage')
    ax.fill_between(times, 0, woo_values, where=(np.array(woo_values) < 0), 
                    alpha=0.3, color='red', label='Defense Advantage')
    
    ax.set_xlabel('Time (seconds)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Window of Opportunity (WOO)', fontsize=12, fontweight='bold')
    ax.set_title('Window of Opportunity Evolution Over Time', fontsize=14, fontweight='bold', pad=15)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()

In [ ]:
def plot_interceptability_field(player_if_fields, ball_land_x, ball_land_y, frame_id, save_path=None):
    """Plot IF field heatmap for players."""
    if not player_if_fields:
        print("No IF fields to plot")
        return
    
    num_players = len(player_if_fields)
    cols = min(3, num_players)
    rows = (num_players + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
    if num_players == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes if isinstance(axes, list) else [axes]
    else:
        axes = axes.flatten()
    
    for idx, (nfl_id, if_data) in enumerate(player_if_fields.items()):
        ax = axes[idx] if num_players > 1 else axes[0]
        
        X = if_data['X']
        Y = if_data['Y']
        IF = if_data['IF']
        
        im = ax.contourf(X, Y, IF, levels=20, cmap='RdYlGn', alpha=0.8)
        ax.contour(X, Y, IF, levels=10, colors='black', linewidths=0.5, alpha=0.3)
        
        # Mark ball landing point
        ax.plot(ball_land_x, ball_land_y, 'k*', markersize=15, label='Ball Landing')
        
        # Mark player position
        ax.plot(if_data['player_x'], if_data['player_y'], 'wo', markersize=10, 
               markeredgecolor='black', markeredgewidth=2, label='Player')
        
        ax.set_xlabel('X (yards)', fontsize=10)
        ax.set_ylabel('Y (yards)', fontsize=10)
        ax.set_title(f"{if_data['player_name']} - IF Field (Frame {frame_id})", fontsize=11, fontweight='bold')
        ax.legend(fontsize=8)
        ax.set_aspect('equal')
        
        plt.colorbar(im, ax=ax, label='IF Probability')
    
    # Hide unused subplots
    for idx in range(num_players, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()

## 9. Main Analysis Pipeline

In [ ]:
def analyze_play(input_file, game_id, play_id, sample_frames=None):
    """
    Complete analysis pipeline for a single play.
    """
    # Load data
    df, _ = load_play_data(input_file)
    play_df, ball_land_x, ball_land_y, num_frames, target_nfl_id, all_players = \
        get_play_data(df, game_id, play_id)
    
    print(f"Analyzing play: Game {game_id}, Play {play_id}")
    print(f"Ball landing: ({ball_land_x:.2f}, {ball_land_y:.2f})")
    print(f"Target player: {all_players.loc[target_nfl_id, 'player_name']}")
    print(f"Number of frames: {num_frames}")
    
    # Estimate ball trajectory
    times, ball_x, ball_y = estimate_ball_trajectory(
        1, ball_land_x, ball_land_y, num_frames
    )
    
    # Calculate WOO over time
    woo_values = []
    receiver_ifs = []
    defender_ifs = []
    frame_ids = []
    
    # Determine which frames to analyze
    if sample_frames is None:
        # Analyze every 3rd frame for efficiency
        frames_to_analyze = range(1, num_frames + 1, 3)
    else:
        frames_to_analyze = sample_frames
    
    for frame_id in frames_to_analyze:
        woo, receiver_if, defender_if = calculate_woo(
            play_df, frame_id, ball_land_x, ball_land_y, 
            times, ball_x, ball_y
        )
        
        if woo is not None:
            woo_values.append(woo)
            receiver_ifs.append(receiver_if)
            defender_ifs.append(defender_if)
            frame_ids.append(frame_id)
    
    # Calculate enhanced metrics
    times_sampled = [(f-1) * DT for f in frame_ids]
    enhanced_metrics = calculate_enhanced_metrics(woo_values, receiver_ifs, defender_ifs,
                                                  frame_ids, times_sampled)
    
    results = {
        'woo_values': woo_values,
        'receiver_ifs': receiver_ifs,
        'defender_ifs': defender_ifs,
        'frame_ids': frame_ids,
        'enhanced_metrics': enhanced_metrics
    }
    
    return results, play_df, times, ball_x, ball_y

## 10. Example Analysis

In [ ]:
print("=" * 60)
print("EXAMPLE 1: Single Play Analysis")
print("=" * 60)
print("Using data from train/ folder: train/input_2023_w01.csv")
print("=" * 60)

input_file = 'train/input_2023_w01.csv'
game_id = 2023090700
play_id = 101

results, play_df, times, ball_x, ball_y = analyze_play(input_file, game_id, play_id)

print("\n" + "=" * 60)
print("ENHANCED METRICS:")
print("=" * 60)
if results['enhanced_metrics']:
    metrics = results['enhanced_metrics']
    print(f"WOO Peak: {metrics['woo_peak']:.3f} (at {metrics['woo_peak_time']:.2f}s)")
    print(f"WOO Initial: {metrics['woo_initial']:.3f}")
    print(f"WOO Final: {metrics['woo_final']:.3f}")
    print(f"Closure Time: {metrics['woo_closure_time']:.2f}s" if metrics['woo_closure_time'] != float('inf') else "Closure Time: Never closed")
    print(f"Window Trend: {metrics['window_trend']}")
    print(f"Receiver IF Δ: {metrics['if_delta_receiver']:+.3f}")
    print(f"Defender IF Δ: {metrics['if_delta_defender']:+.3f}")

# Plot WOO timeseries
plot_woo_timeseries(results['woo_values'], results['frame_ids'], 
                   save_path='example_woo_timeseries.png')

In [ ]:
# Compute and plot IF for a specific frame
frame_id = 10
play_df, ball_land_x, ball_land_y, num_frames, target_nfl_id, all_players = \
    get_play_data(pd.read_csv(input_file), game_id, play_id)

times, ball_x, ball_y = estimate_ball_trajectory(1, ball_land_x, ball_land_y, num_frames)

player_if_fields = compute_interceptability_field(
    play_df, frame_id, ball_land_x, ball_land_y, times, ball_x, ball_y
)

if player_if_fields:
    plot_interceptability_field(player_if_fields, ball_land_x, ball_land_y, 
                               frame_id, save_path='if_field_example.png')